In [1]:
import os
import json
import numpy as np

from rmgpy.molecule.molecule import *
from rmgpy.species import *
from rmgpy.data.rmg import RMGDatabase
from rmgpy.species import Species
from rmgpy import settings

# Load the Mechanism Uncertainty Information

In [2]:
# Make sure you've loaded:
# RMG-Py/save_sensitivity (now you have to rebuild because you turned word-wrap off in chemkin)
# RMG-database/bm_fit_sensitivity

# Load an example ethane model generated using the trees
from rmgpy.tools.uncertainty import Uncertainty, get_rmg_rxn_from_yaml_rxn

chem = '/home/moon/autoscience/uncertainty/partial_nheptane/nheptane_mech_main/chem_annotated.inp'
spec = '/home/moon/autoscience/uncertainty/partial_nheptane/nheptane_mech_main/species_dictionary.txt'
uncertainty = Uncertainty(output_directory='./temp/uncertainty')
uncertainty.load_model(chem, spec)

In [3]:
# Load the database used to generate the model
uncertainty.load_database(
    thermo_libraries=['BurkeH2O2', 'CurranPentane','FFCM1(-)','primaryThermoLibrary', 'thermo_DFT_CCSDTF12_BAC', 'DFT_QCI_thermo', 'CBS_QB3_1dHR'],
    kinetics_families='default',
    reaction_libraries=['CurranPentane','FFCM1(-)','combustion_core/version5'],
    kinetics_depositories=['training'],
)


In [4]:
# Get the different reaction (and thermo- don't forget thermo) sources
uncertainty.extract_sources_from_model()
uncertainty.assign_parameter_uncertainties()

In [5]:
import json
def unpack_sensitivity(long_desc):
    start_str = 'sensitivities = '
    if start_str not in long_desc:
        return []
    start_index = long_desc.find(start_str) + len(start_str)
    sensitivities_str = long_desc[start_index:].replace("'", '"')
    sensitivities_str = sensitivities_str.replace("nan", '"-9999999"')
    sensitivities_str = sensitivities_str.replace('name', 'training_rxn_name')
    return json.loads(sensitivities_str)

In [6]:
# List the reactions that came from nodes on the new trees and collect the families
print('These reactions were estimated using the new trees:')

families_to_lookup = set()

empty_derivatives = 0
for reaction_key in uncertainty.reaction_sources_dict.keys():
    if 'Rate Rules' in uncertainty.reaction_sources_dict[reaction_key].keys():
        family = reaction_key.family
        src = uncertainty.reaction_sources_dict[reaction_key]['Rate Rules'][1]
        if src['node'] != '':
            # look up the node name in the database
            if src['node'] not in uncertainty.database.kinetics.families[family].rules.entries.keys():
                print('NODE NOT FOUND\t', src['node'])
                continue
            
            entry = uncertainty.database.kinetics.families[family].rules.entries[src['node']][0]
            derivatives = unpack_sensitivity(entry.long_desc)
            if derivatives == []:
                # It's okay- I checked and I just haven't generated the derivatives for those families yet
                continue
            
            print(f'{reaction_key}\t({family}) \n\t{src["node"]}')
            print()
            
            # collect the set of families to lookup
            families_to_lookup.add(str(family))


print(families_to_lookup)


These reactions were estimated using the new trees:
C2H5(44) + C5H11(421) <=> C7H16(1)	(R_Recombination) 
	Root_N-1R->H_N-1CNOS->N_N-1COS->O_1CS->C_N-1C-inRing_Ext-2R-R_Ext-3R!H-R_N-Sp-3R!H=2R

NC3H7(93) + PC4H9(183) <=> C7H16(1)	(R_Recombination) 
	Root_N-1R->H_N-1CNOS->N_N-1COS->O_1CS->C_N-1C-inRing_Ext-2R-R_Ext-3R!H-R_N-Sp-3R!H=2R

H(14) + C7H15(682) <=> C7H16(1)	(R_Recombination) 
	Root_1R->H_N-2R->S_N-2CHNO->H_N-2CNO-inRing_Ext-2CNO-R_N-Sp-3R!H=2CCNNOO_N-2CNO->O_Ext-2CN-R

C2H5(44) + C7H15(682) <=> C2H4(11) + C7H16(1)	(Disproportionation) 
	Root_N-4R->H_4CNOS-u1_N-1R!H->O_N-4CNOS->O_Ext-4CNS-R_N-Sp-5R!H#4CCCNNNSSS_N-2R!H->S_N-5R!H->O_Sp-5CS-4CCNSS_Ext-4CNS-R

NC3H7(93) + C7H15(682) <=> C3H6(12) + C7H16(1)	(Disproportionation) 
	Root_Ext-1R!H-R_N-4R->O_N-Sp-5R!H=1R!H_Ext-4CHNS-R_N-6R!H->S_4CHNS->C_N-Sp-6BrBrBrCCCClClClFFFIIINNNOOOPPPSiSiSi#4C_6BrCClFINOPSi->C_N-1R!H-inRing_Ext-4C-R_2R!H->C

O2(2) + C7H15(682) <=> S(763)	(R_Recombination) 
	Root_N-1R->H_N-1CNOS->N_1COS->O_Ext-1O-R_E

In [7]:
print('# Node\tTraining Reaction\t\tdA_node/dA_train\tdE0_node/dA_train\tdn_node/dA_train')
for family in families_to_lookup:
    templateRxnMap = uncertainty.database.kinetics.families[family].get_reaction_matches(thermo_database=uncertainty.database.thermo, remove_degeneracy=True,
                                                                             get_reverse=True, exact_matches_only=False, fix_labels=True)
    
    for r, mechanism_reaction in enumerate(uncertainty.reaction_sources_dict.keys()):
        if mechanism_reaction.__class__.__name__ == 'PDepReaction':
            continue
        if mechanism_reaction.family != family:
            continue
            
        if 'Rate Rules' in uncertainty.reaction_sources_dict[mechanism_reaction].keys():
            # family = reaction_key.family
            src = uncertainty.reaction_sources_dict[mechanism_reaction]['Rate Rules'][1]
            node_name = src['node']
            if node_name != '':
                print(f'{mechanism_reaction.index}\t {mechanism_reaction} \n{node_name}')
                # look up the node name in the database
                if node_name not in uncertainty.database.kinetics.families[family].rules.entries.keys():
                    print('DERIVATIVE NOT YET COMPUTED\t',src['node'])
                    continue
            
                entry = uncertainty.database.kinetics.families[family].rules.entries[node_name][0]
                derivatives = unpack_sensitivity(entry.long_desc)
                
                for i, rxn in enumerate(templateRxnMap[node_name]):
                    N = 35
                    pad_length = N - len(str(rxn))
                    rxn_str = str(rxn) + " " * pad_length
                    #compare names from both sources to check order
                    #print("\n\t", derivatives[i]['name'],"\n\t", rxn_str)
                    dA = derivatives[i]['dA']
                    dE0 = derivatives[i]['dE0']
                    dn = derivatives[i]['dn']
                    if (dn == "-9999999"):
                        dn = np.nan
                    print('\t', rxn_str, "\t\t{0:.6f}".format(dA),
                          "\t\t{0:.6f}".format(dE0),
                          "\t\t{0:.6f}".format(dn))
                    print()
            else:
                print('No node for reaction ', mechanism_reaction) # this needs to complain more loudly
            
    
    
    

# Node	Training Reaction		dA_node/dA_train	dE0_node/dA_train	dn_node/dA_train
970	 CH2CO(25) + C4H8O(265) <=> S(4643) 
Root
	 CH2O + C2H4 <=> C3H6O               		2.229351 		0.006716 		0.036385

	 CH2O + C2H3O <=> C3H5O2             		13.889591 		0.044670 		0.242070

	 CH2O + C2H4 <=> C3H6O               		2.229391 		0.006717 		0.036386

	 CH2S + C2H4 <=> C3H6S               		3.391677 		0.010500 		0.056889

	 CH2O + C2H4 <=> C3H6O               		2.229255 		0.006716 		0.036384

	 C2H2O + C2H4 <=> C4H6O              		3.657907 		0.011366 		0.061585

200	 C2H5(44) + C5H11(421) <=> C7H16(1) 
Root_N-1R->H_N-1CNOS->N_N-1COS->O_1CS->C_N-1C-inRing_Ext-2R-R_Ext-3R!H-R_N-Sp-3R!H=2R
	 C3H3 + C7H7 <=> C10H10              		0.147443 		0.000000 		-0.006377

	 C3H3 + C3H3 <=> C6H6-3              		0.003963 		0.000000 		-0.005709

	 C3H5 + C3H5 <=> C6H10-2             		0.003963 		0.000000 		-0.005710

	 C3H5 + C2H5 <=> C5H10-2             		0.003969 		0.000000 		-0.005661

	 C3H5 + CH3 <=> C4H8-2 

	 OH + CH3O <=> CH4O2                 		0.999500 		0.000000 		nan

966	 O2(2) + S(2632) <=> S(4599) 
Root_N-1R->H_N-1CNOS->N_1COS->O_Ext-1O-R_Ext-2R-R_Ext-4R!H-R_Sp-4R!H-2R_Sp-5R!H-4R!H
	 [O][O] + [CH2]CC <=> C3H7O2         		0.999500 		0.000000 		-0.000000

968	 O2(2) + S(4612) <=> S(4650) 
Root_N-1R->H_N-1CNOS->N_1COS->O_Ext-1O-R_Ext-2R-R_Ext-4R!H-R_Ext-2R-R_Ext-4R!H-R
	 O2 + C10H7 <=> C10H7O2              		0.999500 		0.000000 		0.000000

992	 C2H3(23) + C5H11(421) <=> C7H14(755) 
Root_N-1R->H_N-1CNOS->N_N-1COS->O_1CS->C_N-1C-inRing_Ext-2R-R_Ext-3R!H-R_N-Sp-3R!H=2R
	 C3H3 + C7H7 <=> C10H10              		0.147443 		0.000000 		-0.006377

	 C3H3 + C3H3 <=> C6H6-3              		0.003963 		0.000000 		-0.005709

	 C3H5 + C3H5 <=> C6H10-2             		0.003963 		0.000000 		-0.005710

	 C3H5 + C2H5 <=> C5H10-2             		0.003969 		0.000000 		-0.005661

	 C3H5 + CH3 <=> C4H8-2               		0.003962 		0.000000 		-0.005713

	 C11H23 + CH3 <=> C12H26             		0.074582 		0.000000 

617	 C7H14(754) <=> C3H6(12) + C4H8(189) 
Root_1R!H->C_2R!H->C_N-5R!H->O_Ext-4R!H-R
	 C6H12 <=> C3H6 + C3H6               		-1.728787 		-0.010316 		-0.081880

	 C7H12 <=> C3H6 + C4H6               		-2.331277 		-0.013539 		-0.107458

	 C8H14O2 <=> C3H6 + C5H8O2           		-1.308702 		-0.008069 		-0.064046

	 C7H12O2 <=> C3H6 + C4H6O2           		0.192032 		-0.000042 		-0.000334

	 C8H14O2-2 <=> C4H8 + C4H6O2         		1.337060 		0.006082 		0.048277

888	 S(3227) <=> C2H4(11) + C4H8(190) 
Root_1R!H->C_2R!H->C_N-5R!H->O
	 C5H10 <=> C3H6 + C2H4               		0.543665 		0.002119 		0.023428

	 C6H12 <=> C3H6 + C3H6               		-0.012879 		-0.000799 		-0.009130

	 C6H12-2 <=> C4H8 + C2H4             		1.237113 		0.005756 		0.063995

	 C7H12 <=> C3H6 + C4H6               		-0.429943 		-0.002986 		-0.033528

	 C8H14O2 <=> C3H6 + C5H8O2           		0.277426 		0.000723 		0.007853

	 C7H12O2 <=> C3H6 + C4H6O2           		1.315343 		0.006166 		0.068572

	 C8H14O2-2 <=> C4H8 + C4H6O2        

	 HO + C4H9-2 <=> H2O + C4H8-2        		0.486658 		0.000000 		4086.350495

536	 OH(15) + SC4H9(184) <=> H2O(8) + C4H8(189) 
Root_Ext-2R!H-R_2R!H->C_N-4R->C
	 C4H7-2 + O2 <=> HO2 + C4H6          		0.999500 		0.000000 		nan

537	 C2H5(44) + S(1048) <=> C2H4(11) + S(1049) 
Root_N-4R->H_4CNOS-u1_N-1R!H->O_4CNOS->O_Ext-4O-R_5R!H-u0
	 HO2-2 + HN2 <=> H2O2 + N2           		0.199479 		0.000000 		0.000035

	 HO2-2 + H2N2 <=> H2O2 + HN2         		0.199412 		0.000000 		0.000041

	 HO2-2 + H3N2 <=> H2O2 + H2N2-2      		0.199412 		0.000000 		0.000041

	 HO2-2 + H2NO <=> H2O2 + HNO-2       		0.199866 		0.000000 		0.000003

	 HO2-2 + CH2N <=> H2O2 + CHN         		0.199440 		0.000000 		0.000038

538	 C2H5(44) + S(1025) <=> C2H4(11) + S(1049) 
Root_N-4R->H_4CNOS-u1_N-1R!H->O_4CNOS->O_Ext-4O-R_5R!H-u0
	 HO2-2 + HN2 <=> H2O2 + N2           		0.199479 		0.000000 		0.000035

	 HO2-2 + H2N2 <=> H2O2 + HN2         		0.199412 		0.000000 		0.000041

	 HO2-2 + H3N2 <=> H2O2 + H2N2-2      		0.199412 		0.000000 	

	 C3H7-2 + C4H9 <=> C4H10 + C3H6-2    		0.499784 		0.000000 		0.000009

662	 C7H15(682) + S(781) <=> C7H14(754) + S(785) 
Root_Ext-1R!H-R_N-4R->O_N-Sp-5R!H=1R!H_Ext-4CHNS-R_N-6R!H->S_4CHNS->C_N-Sp-6BrBrBrCCCClClClFFFIIINNNOOOPPPSiSiSi#4C_6BrCClFINOPSi->C_N-1R!H-inRing_Ext-4C-R_2R!H->C
	 C3H7-2 + C3H7 <=> C3H8 + C3H6-2     		0.499732 		0.000000 		-0.000005

	 C3H7-2 + C4H9 <=> C4H10 + C3H6-2    		0.499784 		0.000000 		0.000009

663	 C7H15(679) + S(781) <=> C7H14(754) + S(785) 
Root_Ext-1R!H-R_N-4R->O_N-Sp-5R!H=1R!H_Ext-4CHNS-R_N-6R!H->S_4CHNS->C_N-Sp-6BrBrBrCCCClClClFFFIIINNNOOOPPPSiSiSi#4C_6BrCClFINOPSi->C_N-1R!H-inRing_Ext-4C-R_2R!H->C
	 C3H7-2 + C3H7 <=> C3H8 + C3H6-2     		0.499732 		0.000000 		-0.000005

	 C3H7-2 + C4H9 <=> C4H10 + C3H6-2    		0.499784 		0.000000 		0.000009

664	 C7H15(682) + S(816) <=> C7H14(754) + S(837) 
Root_Ext-1R!H-R_4R->O_Ext-4O-R_Sp-5R!H-1R!H_Ext-5R!H-R
	 C4H7-3 + O2 <=> HO2 + C4H6-2        		0.249886 		0.000000 		-72.837615

	 C4H7-3 + O2 <=> HO2 + C4H6-2 

...


	 C2H3NO2 <=> C2H3NO2-2               		0.022794 		-0.000000 		0.000002

	 C2H4N2O2-9 <=> C2H4N2O2-10          		0.022787 		-0.000000 		0.000002

	 C2H4N2O <=> C2H4N2O-2               		0.022788 		-0.000000 		0.000002

	 CH4N2O <=> CH4N2O-2                 		0.022785 		-0.000000 		0.000002

	 C2H4N2O2-11 <=> C2H4N2O2-12         		0.022787 		-0.000000 		0.000002

	 C2H4N2O2-13 <=> C2H4N2O2-14         		0.022781 		-0.000000 		0.000002

	 C2H3NO2-3 <=> C2H3NO2-4             		0.022779 		-0.000000 		0.000002

	 C2H3NO2-3 <=> C2H3NO2-4             		0.022786 		-0.000000 		0.000002

	 C2H4N2O-3 <=> C2H4N2O-4             		0.022836 		-0.000000 		0.000002

	 C2H5N3O-3 <=> C2H5N3O-4             		0.022791 		-0.000000 		0.000002

	 C2H4N2O-5 <=> C2H4N2O-6             		0.022781 		-0.000000 		0.000002

	 CC=O <=> C=CO                       		-0.000089 		-0.000000 		0.000002

	 CCC(C)=O <=> C=C(O)CC               		-0.000089 		-0.000000 		0.000002

	 CC=O <=> C=CO                       		-0.00008

	 C3H6O <=> C3H6O-2                   		0.000000 		-0.000000 		0.000000

	 C2H4N2O2 <=> C2H4N2O2-2             		0.022801 		-0.000000 		0.000002

	 C2H4N2O2-3 <=> C2H4N2O2-4           		0.022788 		-0.000000 		0.000002

	 C2H4N2O2-5 <=> C2H4N2O2-6           		0.022780 		-0.000000 		0.000002

	 C2H4N2O2-7 <=> C2H4N2O2-8           		0.022798 		-0.000000 		0.000002

	 C3H3NO <=> C3H3NO-2                 		0.022775 		-0.000000 		0.000002

	 C3H6N2O <=> C3H6N2O-2               		0.022780 		-0.000000 		0.000002

	 C2H5N3O <=> C2H5N3O-2               		0.022783 		-0.000000 		0.000002

	 C2H3NO2 <=> C2H3NO2-2               		0.022794 		-0.000000 		0.000002

	 C2H4N2O2-9 <=> C2H4N2O2-10          		0.022787 		-0.000000 		0.000002

	 C2H4N2O <=> C2H4N2O-2               		0.022788 		-0.000000 		0.000002

	 CH4N2O <=> CH4N2O-2                 		0.022785 		-0.000000 		0.000002

	 C2H4N2O2-11 <=> C2H4N2O2-12         		0.022787 		-0.000000 		0.000002

	 C2H4N2O2-13 <=> C2H4N2O2-14         		0.022781 		

	 CH4N2O <=> CH4N2O-2                 		0.022785 		-0.000000 		0.000002

	 C2H4N2O2-11 <=> C2H4N2O2-12         		0.022787 		-0.000000 		0.000002

	 C2H4N2O2-13 <=> C2H4N2O2-14         		0.022781 		-0.000000 		0.000002

	 C2H3NO2-3 <=> C2H3NO2-4             		0.022779 		-0.000000 		0.000002

	 C2H3NO2-3 <=> C2H3NO2-4             		0.022786 		-0.000000 		0.000002

	 C2H4N2O-3 <=> C2H4N2O-4             		0.022836 		-0.000000 		0.000002

	 C2H5N3O-3 <=> C2H5N3O-4             		0.022791 		-0.000000 		0.000002

	 C2H4N2O-5 <=> C2H4N2O-6             		0.022781 		-0.000000 		0.000002

	 CC=O <=> C=CO                       		-0.000089 		-0.000000 		0.000002

	 CCC(C)=O <=> C=C(O)CC               		-0.000089 		-0.000000 		0.000002

	 CC=O <=> C=CO                       		-0.000089 		-0.000000 		0.000002

	 SC=O <=> OC=S                       		0.003895 		-0.000000 		0.000002

	 CC(=O)S <=> CC(=S)O                 		0.003895 		-0.000000 		0.000002

	 CCC(=O)S <=> CCC(=S)O               		0.003895

	 C2H3NO2-3 <=> C2H3NO2-4             		0.022786 		-0.000000 		0.000002

	 C2H4N2O-3 <=> C2H4N2O-4             		0.022836 		-0.000000 		0.000002

	 C2H5N3O-3 <=> C2H5N3O-4             		0.022791 		-0.000000 		0.000002

	 C2H4N2O-5 <=> C2H4N2O-6             		0.022781 		-0.000000 		0.000002

	 CC=O <=> C=CO                       		-0.000089 		-0.000000 		0.000002

	 CCC(C)=O <=> C=C(O)CC               		-0.000089 		-0.000000 		0.000002

	 CC=O <=> C=CO                       		-0.000089 		-0.000000 		0.000002

	 SC=O <=> OC=S                       		0.003895 		-0.000000 		0.000002

	 CC(=O)S <=> CC(=S)O                 		0.003895 		-0.000000 		0.000002

	 CCC(=O)S <=> CCC(=S)O               		0.003895 		-0.000000 		0.000002

	 CCC=O <=> C=COC                     		-0.000088 		-0.000000 		0.000002

	 O=CNC(=O)N <=> NC(=NC=O)O           		0.022802 		-0.000000 		0.000002

	 O=CNC(=O)N <=> OC(=N)NC=O           		0.022787 		-0.000000 		0.000002

	 O=CNC(=O)N <=> NC(=O)N=CO           		0.02278

	 N=CNC=O <=> OC=NC=N                 		0.022785 		-0.000000 		0.000002

	 NC(=O)N <=> NC(=N)O                 		0.022794 		-0.000000 		0.000002

	 O=CNC(=O)N <=> NC(=O)OC=N           		0.022787 		-0.000000 		0.000002

	 O=CNC(=O)N <=> NC(=N)OC=O           		0.022783 		-0.000000 		0.000002

	 O=CNC=O <=> N=COC=O                 		0.022781 		-0.000000 		0.000002

	 O=CN1C=N1 <=> c1ncno1               		0.022776 		-0.000000 		0.000002

	 O=CNC=O <=> N=COC=O                 		0.022763 		-0.000000 		0.000002

	 N=CNC=O <=> N=COC=N                 		0.022788 		-0.000000 		0.000002

	 O=CN1C=C1 <=> c1ncco1               		0.022783 		-0.000000 		0.000002

	 N=CN1C=N1 <=> [nH]1cnnc1            		0.022781 		-0.000000 		0.000002

	 N=CN1C=C1 <=> c1ncc[nH]1            		0.022794 		-0.000000 		0.000002

	 NC(=O)NC=N <=> NC=NC(=O)N           		0.022784 		-0.000000 		0.000002

	 N=CNC=O <=> NC=NC=O                 		0.022786 		-0.000000 		0.000002

	 N#CC(=N)N1C=C1 <=> N#Cc1ncc[nH]1    		0.022755 		

# Save the Mechanism Uncertainty Information

In [8]:
import yaml
import cantera as ct
import pickle
import numpy as np

In [9]:
# see if we can use the reaction as the dictionary key
mechanism_yaml = '/home/moon/autoscience/uncertainty/partial_nheptane/nheptane_mech_main/nheptane.yaml'
gas1 = ct.Solution(mechanism_yaml)
len(gas1.reactions())

2266

In [10]:
# Read in the yaml mechanism file
with open(mechanism_yaml, 'r') as file:
    mech = yaml.load(file, Loader=yaml.FullLoader)
    # print(mech)

In [11]:
auto_gen_families = {}
for family_name in uncertainty.database.kinetics.families.keys():
    if uncertainty.database.kinetics.families[family_name].auto_generated and family_name not in auto_gen_families.keys():
        auto_gen_families[family_name] = uncertainty.database.kinetics.families[family_name].rules.get_entries()
        auto_gen_families[f'{family_name}_labels'] = [entry.label for entry in uncertainty.database.kinetics.families[family_name].rules.get_entries()]
        auto_gen_families[f'{family_name}_rxn_map'] = uncertainty.database.kinetics.families[family_name].get_reaction_matches(thermo_database=uncertainty.database.thermo, remove_degeneracy=True,
                                                                             get_reverse=True, exact_matches_only=False, fix_labels=True)
# print(auto_gen_families)
#if uncertainty.database.kinetics.families[family_name].auto_generated:

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([82.3338,112.396,135.949,158.789,193.294,214.244,237.75],'J/(mol*K)'), H298=(97.8479,'kJ/mol'), S298=(273.009,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(232.805,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(N3s-NCH) + group(N3d-N3sCd) + group(N3d-CC) + group(CO-NN3dOd) + group(Cd-N3dCdH) + group(Cd-N3dCdH) + ring(Cyclohexane)""").
The thermo for this species is probably wrong! Setting CpInf = Cphigh for Entropy calculationat T = 2000.0 K...
ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([82.3338,112.396,135.949,158.789,193.294,214.244,237.75],'J/(mol*K)'), H298=(97.8479,'kJ/mol'), S298=(273.009,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(232.805,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(N3s-NCH) + group(N3d-N3sCd) + group(N3d-CC) + group(CO-NN3dOd) + group(Cd-N3dCdH) + group(Cd-N3dCdH) + ring(Cyclohexane)""").
The thermo for this species is probably wr

In [12]:
for i, reaction in enumerate(mech['reactions']):
    rmg_rxn = get_rmg_rxn_from_yaml_rxn(uncertainty.reaction_list, reaction)
    # print(uncertainty.reaction_sources_dict[rmg_rxn])
    
    rxn_uncertainty = {}
    # Note the source- assume only one source per reaction, or else this might overwrite existing reaction
    for src_key in uncertainty.reaction_sources_dict[rmg_rxn].keys():
        rxn_uncertainty['source'] = src_key
    rxn_uncertainty['value'] = float(uncertainty.kinetic_input_uncertainties[uncertainty.reaction_list.index(rmg_rxn)])
    if src_key == 'Rate Rules':
        family_name = str(rmg_rxn.family)
        src = uncertainty.reaction_sources_dict[rmg_rxn]['Rate Rules'][1]
        new_trees = (src['node'] != '')
        if new_trees:
            rxn_uncertainty['rule_node'] = src['node']
            rule_index = auto_gen_families[family_name + '_labels'].index(src['node'])
            rule = auto_gen_families[family_name][rule_index]
            derivatives = unpack_sensitivity(rule.long_desc)
            
            
            # check that the sensitivities correspond to the correct training reactions
            if len(auto_gen_families[family_name + '_rxn_map'][src['node']]) != len(derivatives):
                print('mismatch', src['node'], family_name)
                
            rxn_uncertainty['sensitivities'] = derivatives
            for j in range(0, len(derivatives)):
                try:
                    t_rxn = auto_gen_families[family_name + '_rxn_map'][src['node']][j].to_cantera()
                except:
                    print(family_name, auto_gen_families[family_name + '_rxn_map'][src['node']][j])
                    continue
                derivatives[j]['training_name'] = t_rxn.equation
                derivatives[j]['training_rate'] = {'A': t_rxn.rate.pre_exponential_factor, 'b': t_rxn.rate.temperature_exponent, 'E': t_rxn.rate.activation_energy}
            # match the training reaction to the actual object
        
    
    mech['reactions'][i]['uncertainty'] = rxn_uncertainty
    if i > 500:
        break

In [13]:
r1 = auto_gen_families[f'Disproportionation_rxn_map']['Root_Ext-2R!H-R'][0].to_cantera()
print(dir(r1.rate))
#print(dir(r1))

if not isinstance(r1, ct._cantera.ElementaryReaction):
    print('Not yet implemented')


reaction1 = {}
for prop in dir(r1):
    if prop[0] == '_':
        continue
    if callable(getattr(r1, prop)):
        continue
    reaction1[prop] = getattr(r1, prop)
reaction1['rate'] = {'A': r1.rate.pre_exponential_factor, 'b': r1.rate.temperature_exponent, 'E': r1.rate.activation_energy}

reaction1


['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', 'activation_energy', 'pre_exponential_factor', 'temperature_exponent']


{'ID': '-1',
 'allow_negative_orders': False,
 'allow_negative_pre_exponential_factor': False,
 'allow_nonreactant_orders': False,
 'duplicate': False,
 'equation': 'C4H7-2 + O2 <=> C4H6 + HO2',
 'orders': {},
 'product_string': 'C4H6 + HO2',
 'products': {'C4H6': 1.0, 'HO2': 1.0},
 'reactant_string': 'C4H7-2 + O2',
 'reactants': {'C4H7-2': 1.0, 'O2': 1.0},
 'reaction_type': 1,
 'reversible': True,
 'rate': {'A': 7230000000.000001, 'b': 0.0, 'E': 92048000.0}}

In [14]:
output_path = '/home/moon/autoscience/uncertainty/partial_nheptane/nheptane_uncertainty.yaml'
with open(output_path, 'w') as file:
    output = yaml.dump(mech, file)
    print(output)

None


In [ ]:
# check for duplicate names - yes there are duplicate equations
reaction_names = []
reaction_IDs = []
for reaction_key in uncertainty.reaction_sources_dict.keys():
    try:
        out_reaction = reaction_key.to_cantera()
        if type(out_reaction) is list:
            out_reaction = out_reaction[0]
        if out_reaction.equation in reaction_names:
            print(f'Duplicate: {out_reaction.equation}')
        reaction_names.append(out_reaction.equation)
        reaction_IDs.append(out_reaction.ID)
    except AttributeError:
        print(f'Failed to convert {reaction_key}')
        print(reaction_key.kinetics)
        # raise
print('\nReaction equations')
print(len(reaction_names))
print(len(set(reaction_names)))
print('\nReaction IDs')
print(len(reaction_IDs))
print(len(set(reaction_IDs)))

In [ ]:
print(len(uncertainty.kinetic_input_uncertainties))
print(len(uncertainty.reaction_sources_dict.keys()))

In [ ]:
#print(mech['reactions'][0])
for i, reaction_key in enumerate(uncertainty.reaction_sources_dict.keys()):
    j = reaction_key.index
    try:
        rxn = reaction_key.to_cantera()
        if type(rxn) is not list:
            rxn = [rxn]
        for r in rxn:
            print(j, r.ID)
            print(reaction_key)
            print()
    except AttributeError:
        print(f'failed to convert {reactions_key}')
    if i>10:
        break
#for rxn in mech['reactions']:
#    print(rxn['equation'])
#    print(uncertainty.reactions)

In [ ]:
#for reaction_key in uncertainty.reaction_sources_dict.keys():
dir(uncertainty)

In [ ]:
print(uncertainty.kinetic_input_uncertainties)
#len(uncertainty.reaction_list)